In [1]:
import pandas as pd
import numpy as np
import ast

# Load the dataset
file_path = "TRADES_CopyTr_90D_ROI.csv"  
df = pd.read_csv(file_path)

# Handle missing values in Trade_History
df["Trade_History"] = df["Trade_History"].fillna("[]"
df["Trade_History"] = df["Trade_History"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract trade details
trade_records = []
for _, row in df.iterrows():
    port_id = row["Port_IDs"]
    trade_history = row["Trade_History"]
    
    if not isinstance(trade_history, list):
        continue  # Skip invalid entries
    
    for trade in trade_history:
        trade_records.append({
            "Port_ID": port_id,
            "Timestamp": trade.get("time", None),
            "Asset": trade.get("symbol", None),
            "Side": trade.get("side", None),
            "PositionSide": trade.get("positionSide", None),
            "Price": trade.get("price", 0),
            "Quantity": trade.get("quantity", 0),
            "Qty": trade.get("qty", 0),
            "RealizedProfit": trade.get("realizedProfit", 0)
        })

# Create structured DataFrame
trade_df = pd.DataFrame(trade_records)

# Calculate financial metrics
def calculate_metrics(trades):
    total_pnl = trades["RealizedProfit"].sum()
    initial_balance = trades["Quantity"].sum()
    roi = (total_pnl / initial_balance) * 100 if initial_balance != 0 else 0
    
    win_positions = (trades["RealizedProfit"] > 0).sum()
    total_positions = len(trades)
    win_rate = (win_positions / total_positions) * 100 if total_positions != 0 else 0
    
    daily_returns = trades.groupby("Timestamp")["RealizedProfit"].sum()
    sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252) if daily_returns.std() != 0 else 0
    
    cumulative_pnl = trades["RealizedProfit"].cumsum()
    peak = cumulative_pnl.cummax()
    max_drawdown = (peak - cumulative_pnl).max()
    
    return pd.Series({
        "ROI": roi,
        "PnL": total_pnl,
        "Sharpe_Ratio": sharpe_ratio,
        "Max_Drawdown": max_drawdown,
        "Win_Rate": win_rate,
        "Win_Positions": win_positions,
        "Total_Positions": total_positions
    })

# Aggregate metrics per account
account_metrics = trade_df.groupby("Port_ID").apply(calculate_metrics).reset_index()

# Ranking system with weighted scores
weights = np.array([0.3, 0.2, 0.2, -0.15, 0.15, 0, 0])  
account_metrics["Score"] = np.dot(account_metrics.iloc[:, 1:8], weights)

# Rank accounts
account_metrics.sort_values(by="Score", ascending=False, inplace=True)

# Save results
account_metrics.to_csv("account_metrics.csv", index=False)
top_20 = account_metrics.head(20)
top_20.to_csv("top_20_accounts.csv", index=False)

print("Analysis complete. Metrics saved.")


Analysis complete. Metrics saved.


In [2]:
print(top_20[["Port_ID", "Score", "ROI", "PnL", "Sharpe_Ratio", "Win_Rate"]])


                 Port_ID         Score       ROI           PnL  Sharpe_Ratio  \
96   4020204877254599680  11204.463216  0.311561  71998.855953      1.824567   
75   3999240873283311617   8478.940651  0.543673  42574.473679      5.782966   
98   4021669203289716224   4909.965682  0.404420  26427.331592      1.746757   
50   3960874214179953664   3587.061356  0.233296  19567.471286      4.309266   
107  4028701921959171840   3441.220810  0.406480  17601.401398      2.630605   
17   3907081197088384000   3304.338252  1.099242  18015.997370      3.527510   
62   3986814617275053313   3280.265306  0.871628  16337.461881      5.677377   
48   3956076827719377409   3271.094804  1.032230  16790.012238      1.532663   
99   4022565861939831809   2790.877137  0.233080  14197.577076      5.691188   
5    3788465932399412480   2533.673576  0.836806  13960.966457      1.855269   
84   4008711265867865600   2191.084144  0.308240  11298.706390      6.815487   
30   3936410995029308417   2142.630605  